### Prova scaricamento dataset con granularità di 1 ora da Coingecko

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import requests
import time

# Imposta le date di inizio e fine
data2 = datetime(2024, 3, 11)
data1 = datetime(2013, 1, 1)

# Inizializza un DataFrame vuoto per contenere i dati
df_generico = pd.DataFrame()

while data1 < data2:
    # Calcola i secondi dall'epoca Unix
    secondi_inizio = str(int((data1 - datetime(1970, 1, 1)).total_seconds()))
    secondi_fine = str(int(((data1 + timedelta(days=90)) - datetime(1970, 1, 1)).total_seconds()))

    # Imposta l'URL e i parametri della richiesta
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range"
    params = {'vs_currency': 'eur', 'from': secondi_inizio, 'to': secondi_fine}

    # Esegui la richiesta
    response = requests.get(url, params=params)
    data = response.json()

    # Preprocessa i dati
    prices = data['prices']
    timestamps = [x[0] for x in prices]
    values = [x[1] for x in prices]

    # Converti i dati pre-elaborati in un DataFrame
    df_temp = pd.DataFrame({'Timestamp': timestamps, 'Price': values})

    # Converti i timestamp in un formato leggibile
    df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'], unit='ms')

    # Concatena i dati raccolti al DataFrame esistente
    df_generico = pd.concat([df_generico, df_temp])

    # Aggiorna la data di inizio per la prossima chiamata
    data1 += timedelta(days=90)

    # Aspetta 12 secondi prima di fare la prossima chiamata per rispettare il limite di rate limit
    print('Aspetto 12 secondi...')
    time.sleep(12)
    print('Riparto!')

# df_generico = df_generico.sort_values('Timestamp')
df_generico

c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


KeyError: 'prices'

In [ ]:
df_generico[1881:]

,Timestamp,Price
90,2018-06-04 00:00:00.000,6548.981060
0,2018-06-04 00:46:43.104,6566.003436
1,2018-06-04 01:46:44.235,6535.370440
2,2018-06-04 02:46:43.178,6535.811201
3,2018-06-04 03:46:45.050,6536.343937
...,...,...
925,2024-03-12 13:00:05.863,65952.669462
926,2024-03-12 14:00:34.863,66251.930803
927,2024-03-12 15:00:40.922,65797.453353
928,2024-03-12 16:01:46.369,65658.767471


In [ ]:
df_tagliato = df_generico[1881:].reset_index(drop=True)
df_tagliato

,Timestamp,Price
0,2018-06-04 00:00:00.000,6548.981060
1,2018-06-04 00:46:43.104,6566.003436
2,2018-06-04 01:46:44.235,6535.370440
3,2018-06-04 02:46:43.178,6535.811201
4,2018-06-04 03:46:45.050,6536.343937
...,...,...
50733,2024-03-12 13:00:05.863,65952.669462
50734,2024-03-12 14:00:34.863,66251.930803
50735,2024-03-12 15:00:40.922,65797.453353
50736,2024-03-12 16:01:46.369,65658.767471


In [ ]:
df_tagliato.isna().sum()

Timestamp    0
Price        0
dtype: int64

In [ ]:
# df_tagliato.to_csv('Prezzo Bitcoin (€), granularità circa oraria, periodo dal 04 06 2018 al 12 03 2024.csv', index=False)

### Aggiornamento dataset con yfinance

#### Dati giornalieri

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Leggi il file CSV esistente
data_esistente = pd.read_csv("Bitcoin (€) da yfinance dal 17-09-2014 al 09-05-2024 con granularità giornaliera.csv")

# Trova l'ultima data nel file CSV esistente
ultima_data = pd.to_datetime(data_esistente['Date']).max()

# Calcola la data di inizio per i nuovi dati
data_inizio = (ultima_data + timedelta(days=1)).strftime('%Y-%m-%d')

# Scarica i dati di Bitcoin in EUR
btc = yf.Ticker("BTC-EUR")

# Scarica i dati storici con intervallo giornaliero
data = btc.history(start=data_inizio, end=datetime.now().strftime('%Y-%m-%d'), interval='1d')

# Resetta l'indice
data = data.reset_index()

# Concatena i nuovi dati con quelli esistenti
data_completa = pd.concat([data_esistente, data])

if "Unnamed: 0" in data_completa.columns:
    data_completa.drop(columns='Unnamed: 0', inplace=True)

data_completa

c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2014-09-17 00:00:00+00:00,359.546204,361.468506,351.586884,355.957367,16389166,0.0,0.0
1,2014-09-18 00:00:00+00:00,355.588409,355.505402,319.789459,328.539368,26691849,0.0,0.0
2,2014-09-19 00:00:00+00:00,328.278503,330.936707,298.921021,307.761139,29560103,0.0,0.0
3,2014-09-20 00:00:00+00:00,307.665253,329.978180,303.931244,318.758972,28736826,0.0,0.0
4,2014-09-21 00:00:00+00:00,318.120514,321.504517,306.502197,310.632446,20702625,0.0,0.0
...,...,...,...,...,...,...,...,...
22,2024-06-01 00:00:00+00:00,63115.078125,63479.875000,61443.273438,62161.078125,25224290872,0.0,0.0
23,2024-06-02 00:00:00+00:00,62159.417969,62487.687500,62064.171875,62365.332031,10723063091,0.0,0.0
24,2024-06-03 00:00:00+00:00,62368.406250,63012.160156,62034.824219,62428.628906,15766277926,0.0,0.0
25,2024-06-04 00:00:00+00:00,62430.742188,64715.664062,62274.976562,63082.285156,29706470423,0.0,0.0


In [2]:
# Salva il tutto in un nuovo file CSV con la data odierna nel nome del file
# data_completa.to_csv(f"Bitcoin (€) da yfinance dal 17-09-2014 al {pd.to_datetime(data_completa['Date']).max().strftime('%d-%m-%Y')} con granularità giornaliera.csv", index=False)

#### Dati orari (da aggiornare al massimo ogni 730 giorni perchè si possono scaricare solo i dati degli ultimi 730 giorni)

In [4]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

# Leggi il file CSV esistente
data_esistente = pd.read_csv("Bitcoin (€) da yfinance dal 06-05-2022 alle 21 al 05-05-2024 alle 23 con granularità oraria.csv")

# Trova l'ultima data nel file CSV esistente
ultima_data = pd.to_datetime(data_esistente['Datetime']).max()

# Calcola la data di inizio per i nuovi dati
data_inizio = (ultima_data + timedelta(hours=1)).strftime('%Y-%m-%d')

# Scarica i dati storici con intervallo giornaliero
data_fine = datetime.now().strftime('%Y-%m-%d')

# Scarica i dati di Bitcoin in EUR
btc = yf.Ticker("BTC-EUR")

data = btc.history(start=data_inizio, end=data_fine, interval='1h')

# Resetta l'indice
data = data.reset_index()

# Concatena i nuovi dati con quelli esistenti
data_completa = pd.concat([data_esistente, data])

# Rimuovi la colonna non necessaria
if "Unnamed: 0" in data_completa.columns:
    data_completa.drop(columns='Unnamed: 0', inplace=True)

data_completa

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-05-06 21:00:00+00:00,34211.734375,34218.632812,34080.335938,34112.898438,0,0.0,0.0
1,2022-05-06 22:00:00+00:00,34113.457031,34174.781250,34063.156250,34170.894531,0,0.0,0.0
2,2022-05-06 23:00:00+00:00,34171.371094,34237.242188,34155.128906,34176.558594,0,0.0,0.0
3,2022-05-07 00:00:00+00:00,34169.015625,34182.152344,34109.246094,34143.449219,0,0.0,0.0
4,2022-05-07 01:00:00+00:00,34144.773438,34174.976562,34089.144531,34148.765625,0,0.0,0.0
...,...,...,...,...,...,...,...,...
91,2024-05-09 19:00:00+00:00,57728.093750,57973.621094,57687.101562,57886.734375,0,0.0,0.0
92,2024-05-09 20:00:00+00:00,57895.015625,58083.500000,57846.457031,58083.500000,0,0.0,0.0
93,2024-05-09 21:00:00+00:00,58079.878906,58247.542969,57907.917969,58020.367188,0,0.0,0.0
94,2024-05-09 22:00:00+00:00,58014.769531,58771.429688,57983.117188,58688.980469,157687808,0.0,0.0


In [5]:
# Salva il tutto in un nuovo file CSV con la data odierna nel nome del file
data_completa.to_csv(f"Bitcoin (€) da yfinance dal 06-05-2022 alle 21 al {pd.to_datetime(data_completa['Datetime']).max().strftime('%d-%m-%Y alle %H')} con granularità oraria.csv", index=False)

#### Dati al minuto (da aggiornare al massimo ogni 7 giorni perchè si possono scaricare solo i dati degli ultimi 7 giorni)

In [ ]:
# import pandas as pd
# from datetime import datetime, timedelta
# import yfinance as yf

# # Leggi il file CSV esistente
# data_esistente = pd.read_csv(".csv")

# # Trova l'ultima data nel file CSV esistente
# ultima_data = pd.to_datetime(data_esistente['Datetime']).max()

# # Calcola la data di inizio per i nuovi dati
# data_inizio = (ultima_data + timedelta(minutes=1)).strftime('%Y-%m-%d')

# # Scarica i dati storici con intervallo giornaliero
# data_fine = datetime.now().strftime('%Y-%m-%d')

# # Scarica i dati di Bitcoin in EUR
# btc = yf.Ticker("BTC-EUR")

# data = btc.history(start=data_inizio, end=data_fine, interval='1h')

# # Resetta l'indice
# data = data.reset_index()

# # Concatena i nuovi dati con quelli esistenti
# data_completa = pd.concat([data_esistente, data])

# # Rimuovi la colonna non necessaria
# data_completa.drop(columns='Unnamed: 0', inplace=True)

# data_completa